문장의 특성을 이해하면서 분석하게 된다
### 로이터 뉴스 데이터 로딩 및 탐색 

#### 데이터 로드

In [1]:
from tensorflow.keras.datasets import reuters
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 

In [2]:
(X_train,y_train), (X_test,y_test) = reuters.load_data()

C:\Users\SM2127\anaconda3\envs\deep01\lib\site-packages\tensorflow_core\python\keras\datasets\reuters.py:113: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\SM2127\anaconda3\envs\deep01\lib\site-packages\tensorflow_core\python\keras\datasets\reuters.py:114: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [3]:
X_train.shape, y_train.shape, X_test.shape,y_test.shape

((8982,), (8982,), (2246,), (2246,))

In [4]:
# 정답 종류 확인
# 0 ~ 45 : 46개로 분류된 뉴스 데이터 
np.unique(y_train)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45], dtype=int64)

In [5]:
# 정답 종류별 갯수 확인
pd.Series(y_train).value_counts()

3     3159
4     1949
19     549
16     444
1      432
11     390
20     269
13     172
8      139
10     124
9      101
21     100
25      92
2       74
18      66
24      62
0       55
34      50
36      49
12      49
28      48
6       48
30      45
23      41
31      39
17      39
40      36
32      32
41      30
14      26
39      24
26      24
43      21
15      20
29      19
37      19
38      19
45      18
5       17
7       16
27      15
22      15
42      13
44      12
33      11
35      10
dtype: int64

In [6]:
X_train[0] #뉴스데이터가 토큰화,수치화 진행된 데이터 

[1,
 27595,
 28842,
 8,
 43,
 10,
 447,
 5,
 25,
 207,
 270,
 5,
 3095,
 111,
 16,
 369,
 186,
 90,
 67,
 7,
 89,
 5,
 19,
 102,
 6,
 19,
 124,
 15,
 90,
 67,
 84,
 22,
 482,
 26,
 7,
 48,
 4,
 49,
 8,
 864,
 39,
 209,
 154,
 6,
 151,
 6,
 83,
 11,
 15,
 22,
 155,
 11,
 15,
 7,
 48,
 9,
 4579,
 1005,
 504,
 6,
 258,
 6,
 272,
 11,
 15,
 22,
 134,
 44,
 11,
 15,
 16,
 8,
 197,
 1245,
 90,
 67,
 52,
 29,
 209,
 30,
 32,
 132,
 6,
 109,
 15,
 17,
 12]

- 뉴스 기사를 토큰화, 수치화를 진행한 데이터셋
- 뉴스 기사마다 길이가 다르다 
- 수치화를 진행할 때 단어의 빈도수를 기반으로 수치화 진행
- 숫자가 작을수록 자주 등장한 단어 


In [7]:
reuters.get_word_index()

{'mdbl': 10996,
 'fawc': 16260,
 'degussa': 12089,
 'woods': 8803,
 'hanging': 13796,
 'localized': 20672,
 'sation': 20673,
 'chanthaburi': 20675,
 'refunding': 10997,
 'hermann': 8804,
 'passsengers': 20676,
 'stipulate': 20677,
 'heublein': 8352,
 'screaming': 20713,
 'tcby': 16261,
 'four': 185,
 'grains': 1642,
 'broiler': 20680,
 'wooden': 12090,
 'wednesday': 1220,
 'highveld': 13797,
 'duffour': 7593,
 '0053': 20681,
 'elections': 3914,
 '270': 2563,
 '271': 3551,
 '272': 5113,
 '273': 3552,
 '274': 3400,
 'rudman': 7975,
 '276': 3401,
 '277': 3478,
 '278': 3632,
 '279': 4309,
 'dormancy': 9381,
 'errors': 7247,
 'deferred': 3086,
 'sptnd': 20683,
 'cooking': 8805,
 'stratabit': 20684,
 'designing': 16262,
 'metalurgicos': 20685,
 'databank': 13798,
 '300er': 20686,
 'shocks': 20687,
 'nawg': 7972,
 'tnta': 20688,
 'perforations': 20689,
 'affiliates': 2891,
 '27p': 20690,
 'ching': 16263,
 'china': 595,
 'wagyu': 16264,
 'affiliated': 3189,
 'chino': 16265,
 'chinh': 16266,
 '

#### 학습
- RNN학습을 위해선 뉴스 기사의 길이를 맞춰주어야함
- 길이가 긴 뉴스기사는 단어를 잘라서 길이를 축소
- 길이가 짧은 뉴스기사는 패딩작업(0을 채움)을 이용해 길이를 확장

##### 시퀀스 길이 확인
- 적절한 길이는 무엇일까
- 보편적으로 많은것을 사용

In [8]:
train_len = [ len(doc) for doc in X_train]

In [9]:
print('최대값',max(train_len))
print('최소값',min(train_len))
print('평균값',np.mean(train_len))
print('중앙값',np.median(train_len))

최대값 2376
최소값 13
평균값 145.5398574927633
중앙값 95.0


- 시퀀스 길이를 120정도로 맞춰보자

In [10]:
from tensorflow.keras.preprocessing import sequence

In [11]:
# 길이가 긴건 자르고 짧은건 늘려준다
X_train_seq = sequence.pad_sequences(X_train,maxlen = 120)
X_test_seq = sequence.pad_sequences(X_test,maxlen = 120)

In [12]:
X_train_seq.shape, X_test_seq.shape

((8982, 120), (2246, 120))

In [13]:
X_train_seq[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     1, 27595, 28842,
           8,    43,    10,   447,     5,    25,   207,   270,     5,
        3095,   111,    16,   369,   186,    90,    67,     7,    89,
           5,    19,   102,     6,    19,   124,    15,    90,    67,
          84,    22,   482,    26,     7,    48,     4,    49,     8,
         864,    39,   209,   154,     6,   151,     6,    83,    11,
          15,    22,   155,    11,    15,     7,    48,     9,  4579,
        1005,   504,     6,   258,     6,   272,    11,    15,    22,
         134,    44,    11,    15,    16,     8,   197,  1245,    90,
          67,    52,    29,   209,    30,    32,   132,     6,   109,
          15,    17,    12])

In [14]:
X_train_seq_reshape = X_train_seq.reshape(8982,120,1)
X_test_seq_reshape = X_test_seq.reshape(2246,120,1)

In [15]:
X_train_seq_reshape.shape, X_test_seq_reshape.shape

((8982, 120, 1), (2246, 120, 1))

모델링

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

In [17]:
model1 = Sequential()

model1.add(SimpleRNN(128, input_shape=(120,1))) #120개길이, 단어하나

model1.add(Dense(64,activation = 'relu'))
model1.add(Dense(46,activation = 'softmax')) # 0~45클래스중 하나의 값 출력
                                            #46개의 결과값을 출력하고 각각
                                            #몇퍼센트 해당하는지 알아보기위해서
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 128)               16640     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 46)                2990      
Total params: 27,886
Trainable params: 27,886
Non-trainable params: 0
_________________________________________________________________


In [29]:
model1.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [30]:
import os
# 모델을 저장할 폴더명
MODEL_FOLDER = './model'

# 해당 폴더가 없다면 해당 폴더를 생성
if not os.path.exists(MODEL_FOLDER) :
    os.mkdir(MODEL_FOLDER)

In [31]:
# 저장할 파일 명 설정
 #{epoch:04d} : 반복수를 4자리로 표시
# {val_accuracy:.4f}:검증 정확도를 소수점 4째자리까지 표시
#hdf5 파일형식
modelpath = MODEL_FOLDER + './reuters-{epoch:04d}-{val_accuracy:4f}.hdf5'

In [32]:
# 베스트 모델을 찾아서 만들어둔 파일 명으로 저장
# ModelCheckpoint(filepath = 파일 경로, monitor = 기준값,save_bast_only=True)
#save_bast_only=True : 더 나은 결과값만 저장
mc = ModelCheckpoint(filepath = modelpath,
                    monitor = 'val_accuracy',
                    save_best_only = True,
                    verbose = 1) #verbose=1 진행결과를 보지 않겠다

In [33]:
# EarlyStopping(monitor,patience = 기다리는 횟수)
# patience = 20 : monitor 에 적은 기준에 따라 학습 결과가 더 나아지지 않더라도 
#20번은 돌려보겠다.
# patience가 있어야지만 조금씩 나아진 결과를 확인할 수 있다.
#EarlyStopping 한 번 안좋아지면 멈춤
es = EarlyStopping(monitor='val_accuracy',
                  patience = 20)

In [35]:
h1 = model1.fit(X_train_seq_reshape,y_train,
               validation_split = 0.2,
               epochs = 100,callbacks=[mc,es])

Train on 7185 samples, validate on 1797 samples
Epoch 1/100
7168/7185 [============================>.] - ETA: 0s - loss: 1.9448 - accuracy: 0.4227
Epoch 00001: val_accuracy did not improve from 0.38230
7185/7185 [==============================] - 6s 903us/sample - loss: 1.9448 - accuracy: 0.4225 - val_loss: 2.4979 - val_accuracy: 0.3706
Epoch 2/100
7136/7185 [============================>.] - ETA: 0s - loss: 1.9333 - accuracy: 0.4281
Epoch 00002: val_accuracy did not improve from 0.38230
7185/7185 [==============================] - 7s 959us/sample - loss: 1.9330 - accuracy: 0.4280 - val_loss: 2.5312 - val_accuracy: 0.3734
Epoch 3/100
7168/7185 [============================>.] - ETA: 0s - loss: 1.9330 - accuracy: 0.4251
Epoch 00003: val_accuracy did not improve from 0.38230
7185/7185 [==============================] - 7s 919us/sample - loss: 1.9330 - accuracy: 0.4253 - val_loss: 2.5365 - val_accuracy: 0.3756
Epoch 4/100
7136/7185 [============================>.] - ETA: 0s - loss: 1.9384

Epoch 29/100
7136/7185 [============================>.] - ETA: 0s - loss: 1.8994 - accuracy: 0.4361 ETA: 0s - loss: 1.9015 - accura
Epoch 00029: val_accuracy did not improve from 0.38230
7185/7185 [==============================] - 7s 966us/sample - loss: 1.8974 - accuracy: 0.4373 - val_loss: 2.5875 - val_accuracy: 0.3734
Epoch 30/100
7136/7185 [============================>.] - ETA: 0s - loss: 1.8957 - accuracy: 0.4368
Epoch 00030: val_accuracy did not improve from 0.38230
7185/7185 [==============================] - 7s 966us/sample - loss: 1.8964 - accuracy: 0.4369 - val_loss: 2.6495 - val_accuracy: 0.3734
Epoch 31/100
7136/7185 [============================>.] - ETA: 0s - loss: 1.9106 - accuracy: 0.4351
Epoch 00031: val_accuracy did not improve from 0.38230
7185/7185 [==============================] - 7s 967us/sample - loss: 1.9113 - accuracy: 0.4347 - val_loss: 2.5949 - val_accuracy: 0.3712


#### 모델링2: RNN레이어 추가

In [43]:
model2 = Sequential()

# return_sequeces = True : 뉴런마다 똑같은 데이터가 들어갈 수 있게 한다
model2.add(SimpleRNN(128, input_shape=(120,1),return_sequences = True)) #120개길이, 단어하나
model2.add(SimpleRNN(64, return_sequences = True))
model2.add(SimpleRNN(64))

# 기본적으로 모든 층이 다수입력 단일출력
# RNN을 여러 층을 쌓으려고 생각한다면 각 층을 다수입력 다수출력으로 바꿔야함
# 분류기로 넘겨줄때 특성값을 넘겨줘야 하기 때문에
# return_sequences 가 필요없음

model2.add(Dense(64,activation = 'relu'))
model2.add(Dense(46,activation = 'softmax')) # 0~45클래스중 하나의 값 출력
                                            #46개의 결과값을 출력하고 각각
                                            #몇퍼센트 해당하는지 알아보기위해서
model2.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_15 (SimpleRNN)    (None, 120, 128)          16640     
_________________________________________________________________
simple_rnn_16 (SimpleRNN)    (None, 120, 64)           12352     
_________________________________________________________________
simple_rnn_17 (SimpleRNN)    (None, 64)                8256      
_________________________________________________________________
dense_10 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_11 (Dense)             (None, 46)                2990      
Total params: 44,398
Trainable params: 44,398
Non-trainable params: 0
_________________________________________________________________


In [44]:
model2.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [45]:
h2 = model2.fit(X_train_seq_reshape,y_train,
               validation_split = 0.2,
               epochs = 100,callbacks=[mc,es])

Train on 7185 samples, validate on 1797 samples
Epoch 1/100
7168/7185 [============================>.] - ETA: 0s - loss: 2.4434 - accuracy: 0.3606
Epoch 00001: val_accuracy did not improve from 0.43795
7185/7185 [==============================] - 21s 3ms/sample - loss: 2.4421 - accuracy: 0.3610 - val_loss: 2.3775 - val_accuracy: 0.3667
Epoch 2/100
7168/7185 [============================>.] - ETA: 0s - loss: 2.3309 - accuracy: 0.3906
Epoch 00002: val_accuracy did not improve from 0.43795
7185/7185 [==============================] - 20s 3ms/sample - loss: 2.3307 - accuracy: 0.3908 - val_loss: 2.3891 - val_accuracy: 0.3851
Epoch 3/100
7168/7185 [============================>.] - ETA: 0s - loss: 2.3487 - accuracy: 0.3832
Epoch 00003: val_accuracy did not improve from 0.43795
7185/7185 [==============================] - 20s 3ms/sample - loss: 2.3501 - accuracy: 0.3830 - val_loss: 2.4121 - val_accuracy: 0.3450
Epoch 4/100
7168/7185 [============================>.] - ETA: 0s - loss: 2.3598 - 

Epoch 29/100
7168/7185 [============================>.] - ETA: 0s - loss: 2.0895 - accuracy: 0.4515
Epoch 00029: val_accuracy did not improve from 0.45409
7185/7185 [==============================] - 19s 3ms/sample - loss: 2.0893 - accuracy: 0.4514 - val_loss: 2.2248 - val_accuracy: 0.4374
Epoch 30/100
7168/7185 [============================>.] - ETA: 0s - loss: 2.1060 - accuracy: 0.4406
Epoch 00030: val_accuracy did not improve from 0.45409
7185/7185 [==============================] - 19s 3ms/sample - loss: 2.1052 - accuracy: 0.4405 - val_loss: 2.1832 - val_accuracy: 0.4418
Epoch 31/100
7168/7185 [============================>.] - ETA: 0s - loss: 2.1971 - accuracy: 0.4129
Epoch 00031: val_accuracy did not improve from 0.45409
7185/7185 [==============================] - 19s 3ms/sample - loss: 2.1981 - accuracy: 0.4129 - val_loss: 2.3171 - val_accuracy: 0.3779
Epoch 32/100
7168/7185 [============================>.] - ETA: 0s - loss: 2.1918 - accuracy: 0.4107
Epoch 00032: val_accuracy d

- 다수입력 다수출력 > 영상 프레임마다 폭력이 있는지 없는지 구분
- 다수입력 단일출력 > 영상 내에서 폭력이 있는지 없는지 구분

RNN의 문제점
- 데이터가 길어지면 기억값을 전달하는 과정에서 기억값이 소실이 됨
- LSTM 긴 기간동안 기억을 하는 학습을 수행하는 능력을 가짐 
- 중요한 기억에는 가중치부여, 중요하지 않은 기억에는 강중치를 덜 부여 

##### 모델링3: LSTM 레이어 추가하기(장단기 기억 메모리)
- 시퀀스 길이가 길어질수록 RNN 은 기억값이 소실되는 문제가 발생 
- 기억을 관리하는 LSTM은 이와같은 경우에서 성능을 끌어올릴 수 있다
- 중요한 기억은 강조, 불필요한 기억은 삭제

In [46]:
from tensorflow.keras.layers import LSTM

In [47]:
model3 = Sequential()

# return_sequeces = True : 뉴런마다 똑같은 데이터가 들어갈 수 있게 한다
model3.add(LSTM(128, input_shape=(120,1),return_sequences = True)) #120개길이, 단어하나
model3.add(LSTM(64, return_sequences = True))
model3.add(LSTM(64))

# 기본적으로 모든 층이 다수입력 단일출력
# RNN을 여러 층을 쌓으려고 생각한다면 각 층을 다수입력 다수출력으로 바꿔야함
# 분류기로 넘겨줄때 특성값을 넘겨줘야 하기 때문에
# return_sequences 가 필요없음

model3.add(Dense(64,activation = 'relu'))
model3.add(Dense(46,activation = 'softmax')) # 0~45클래스중 하나의 값 출력
                                            #46개의 결과값을 출력하고 각각
                                            #몇퍼센트 해당하는지 알아보기위해서
model3.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 120, 128)          66560     
_________________________________________________________________
lstm_1 (LSTM)                (None, 120, 64)           49408     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_12 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_13 (Dense)             (None, 46)                2990      
Total params: 156,142
Trainable params: 156,142
Non-trainable params: 0
_________________________________________________________________


In [48]:
model3.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [49]:
h3 = model3.fit(X_train_seq_reshape,y_train,
               validation_split = 0.2,
               epochs =100,
               callbacks = [mc, es])

Train on 7185 samples, validate on 1797 samples
Epoch 1/100
7104/7185 [============================>.] - ETA: 0s - loss: 2.3233 - accuracy: 0.4091
Epoch 00001: val_accuracy improved from 0.45409 to 0.46244, saving model to ./model./reuters-0001-0.462437.hdf5
7185/7185 [==============================] - 9s 1ms/sample - loss: 2.3197 - accuracy: 0.4103 - val_loss: 2.1438 - val_accuracy: 0.4624
Epoch 2/100
7104/7185 [============================>.] - ETA: 0s - loss: 2.1260 - accuracy: 0.4668
Epoch 00002: val_accuracy improved from 0.46244 to 0.47635, saving model to ./model./reuters-0002-0.476349.hdf5
7185/7185 [==============================] - 3s 457us/sample - loss: 2.1245 - accuracy: 0.4678 - val_loss: 2.1294 - val_accuracy: 0.4763
Epoch 3/100
7168/7185 [============================>.] - ETA: 0s - loss: 2.0972 - accuracy: 0.4639
Epoch 00003: val_accuracy did not improve from 0.47635
7185/7185 [==============================] - 3s 448us/sample - loss: 2.0962 - accuracy: 0.4643 - val_los

Epoch 27/100
7072/7185 [============================>.] - ETA: 0s - loss: 1.6797 - accuracy: 0.5431
Epoch 00027: val_accuracy improved from 0.52476 to 0.52643, saving model to ./model./reuters-0027-0.526433.hdf5
7185/7185 [==============================] - 3s 437us/sample - loss: 1.6749 - accuracy: 0.5438 - val_loss: 1.8031 - val_accuracy: 0.5264
Epoch 28/100
7136/7185 [============================>.] - ETA: 0s - loss: 1.6481 - accuracy: 0.5497
Epoch 00028: val_accuracy improved from 0.52643 to 0.53422, saving model to ./model./reuters-0028-0.534224.hdf5
7185/7185 [==============================] - 3s 437us/sample - loss: 1.6493 - accuracy: 0.5493 - val_loss: 1.7916 - val_accuracy: 0.5342
Epoch 29/100
7104/7185 [============================>.] - ETA: 0s - loss: 1.6387 - accuracy: 0.5501
Epoch 00029: val_accuracy did not improve from 0.53422
7185/7185 [==============================] - 3s 435us/sample - loss: 1.6398 - accuracy: 0.5503 - val_loss: 1.7840 - val_accuracy: 0.5192
Epoch 30/1

7072/7185 [============================>.] - ETA: 0s - loss: 1.1703 - accuracy: 0.6671
Epoch 00054: val_accuracy did not improve from 0.54368
7185/7185 [==============================] - 3s 433us/sample - loss: 1.1738 - accuracy: 0.6665 - val_loss: 1.9530 - val_accuracy: 0.5420
Epoch 55/100
7104/7185 [============================>.] - ETA: 0s - loss: 1.1793 - accuracy: 0.6627
Epoch 00055: val_accuracy did not improve from 0.54368
7185/7185 [==============================] - 3s 433us/sample - loss: 1.1808 - accuracy: 0.6622 - val_loss: 2.0218 - val_accuracy: 0.5170
Epoch 56/100
7072/7185 [============================>.] - ETA: 0s - loss: 1.1585 - accuracy: 0.6646
Epoch 00056: val_accuracy did not improve from 0.54368
7185/7185 [==============================] - 3s 434us/sample - loss: 1.1584 - accuracy: 0.6644 - val_loss: 2.0145 - val_accuracy: 0.5159
Epoch 57/100
7040/7185 [============================>.] - ETA: 0s - loss: 1.1391 - accuracy: 0.6744
Epoch 00057: val_accuracy did not imp

##### 모델링 4: 워드임배딩 레이어 추가
- 단순하게 단어를 구분하는 원핫인코딩, 단어의 빈도를 사용하는 수치화방법은 
- 자연어를 해석하기에 무리가 있음

- 단어 사이의 관계를 밀집된 표현으로 만들어줄 수 있는 워드 임베딩 사용

In [52]:
# 워드임베딩
from tensorflow.keras.layers import Embedding

In [51]:
# 사용하는 단어의 수를 제한하자(빈도가 낮은 단어들을 삭제)
(X_train,y_train),(X_test,y_test) = reuters.load_data(num_words = 1000)

C:\Users\SM2127\anaconda3\envs\deep01\lib\site-packages\tensorflow_core\python\keras\datasets\reuters.py:113: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\SM2127\anaconda3\envs\deep01\lib\site-packages\tensorflow_core\python\keras\datasets\reuters.py:114: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [53]:
# 길이가 긴건 자르고 짧은건 늘려준다
X_train_seq = sequence.pad_sequences(X_train,maxlen = 120)
X_test_seq = sequence.pad_sequences(X_test,maxlen = 120)

In [54]:
model4 = Sequential()

# return_sequeces = True : 뉴런마다 똑같은 데이터가 들어갈 수 있게 한다
# 임베딩 레이어는 항상 첫번째 층에 사용
#(사용하는 단어의 수 , 각 단어가 표현될 숫자의 갯수 = feature수)
model4.add(Embedding(1000,50))
model4.add(LSTM(128,return_sequences = True)) #120개길이, 단어하나
model4.add(LSTM(64, return_sequences = True))
model4.add(LSTM(64))

# 기본적으로 모든 층이 다수입력 단일출력
# RNN을 여러 층을 쌓으려고 생각한다면 각 층을 다수입력 다수출력으로 바꿔야함
# 분류기로 넘겨줄때 특성값을 넘겨줘야 하기 때문에
# return_sequences 가 필요없음

model4.add(Dense(64,activation = 'relu'))
model4.add(Dense(46,activation = 'softmax')) # 0~45클래스중 하나의 값 출력
                                            #46개의 결과값을 출력하고 각각
                                            #몇퍼센트 해당하는지 알아보기위해서
model4.summary()

ERROR! Session/line number was not unique in database. History logging moved to new session 603
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 50)          50000     
_________________________________________________________________
lstm_3 (LSTM)                (None, None, 128)         91648     
_________________________________________________________________
lstm_4 (LSTM)                (None, None, 64)          49408     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_14 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_15 (Dense)             (None, 46)                2990      
Total params: 231,230
T

In [55]:
model4.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [56]:
h4 = model4.fit(X_train_seq,y_train, # 임베딩이 자료구조를 변화하기 때문에 
                #reshape를 할 필요가 없음
               validation_split = 0.2,
               epochs =100,
               callbacks = [mc, es])

ERROR! Session/line number was not unique in database. History logging moved to new session 604
Train on 7185 samples, validate on 1797 samples
Epoch 1/100
7104/7185 [============================>.] - ETA: 0s - loss: 2.5065 - accuracy: 0.3487
Epoch 00001: val_accuracy did not improve from 0.54368
7185/7185 [==============================] - 7s 1ms/sample - loss: 2.5027 - accuracy: 0.3491 - val_loss: 2.4259 - val_accuracy: 0.3450
Epoch 2/100
7072/7185 [============================>.] - ETA: 0s - loss: 2.4159 - accuracy: 0.3538
Epoch 00002: val_accuracy did not improve from 0.54368
7185/7185 [==============================] - 4s 490us/sample - loss: 2.4157 - accuracy: 0.3534 - val_loss: 2.4090 - val_accuracy: 0.3450
Epoch 3/100
7072/7185 [============================>.] - ETA: 0s - loss: 2.4142 - accuracy: 0.3544
Epoch 00003: val_accuracy did not improve from 0.54368
7185/7185 [==============================] - 3s 487us/sample - loss: 2.4152 - accuracy: 0.3534 - val_loss: 2.4126 - val_ac

7072/7185 [============================>.] - ETA: 0s - loss: 1.2464 - accuracy: 0.6875
Epoch 00026: val_accuracy did not improve from 0.67112
7185/7185 [==============================] - 3s 483us/sample - loss: 1.2462 - accuracy: 0.6878 - val_loss: 1.4197 - val_accuracy: 0.6617
Epoch 27/100
7072/7185 [============================>.] - ETA: 0s - loss: 1.2131 - accuracy: 0.6926
Epoch 00027: val_accuracy did not improve from 0.67112
7185/7185 [==============================] - 4s 491us/sample - loss: 1.2122 - accuracy: 0.6928 - val_loss: 1.3579 - val_accuracy: 0.6672
Epoch 28/100
7072/7185 [============================>.] - ETA: 0s - loss: 1.1747 - accuracy: 0.7025
Epoch 00028: val_accuracy did not improve from 0.67112
7185/7185 [==============================] - 3s 485us/sample - loss: 1.1749 - accuracy: 0.7029 - val_loss: 1.3446 - val_accuracy: 0.6672
Epoch 29/100
7072/7185 [============================>.] - ETA: 0s - loss: 1.1464 - accuracy: 0.7053
Epoch 00029: val_accuracy improved fr

Epoch 53/100
7072/7185 [============================>.] - ETA: 0s - loss: 0.5914 - accuracy: 0.8498
Epoch 00053: val_accuracy improved from 0.70506 to 0.70673, saving model to ./model./reuters-0053-0.706733.hdf5
7185/7185 [==============================] - 3s 481us/sample - loss: 0.5913 - accuracy: 0.8501 - val_loss: 1.4918 - val_accuracy: 0.7067
Epoch 54/100
7072/7185 [============================>.] - ETA: 0s - loss: 0.6160 - accuracy: 0.8447
Epoch 00054: val_accuracy did not improve from 0.70673
7185/7185 [==============================] - 3s 478us/sample - loss: 0.6151 - accuracy: 0.8450 - val_loss: 1.4977 - val_accuracy: 0.6962
Epoch 55/100
7072/7185 [============================>.] - ETA: 0s - loss: 0.5888 - accuracy: 0.8510
Epoch 00055: val_accuracy did not improve from 0.70673
7185/7185 [==============================] - 3s 478us/sample - loss: 0.5889 - accuracy: 0.8507 - val_loss: 1.4749 - val_accuracy: 0.6989
Epoch 56/100
7072/7185 [============================>.] - ETA: 0s -